In [ ]:
#pip install selenium

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
driver = webdriver.Chrome('C:/Users/Seraphina/Desktop/chromedriver.exe')
driver.get("http://s4u.daegu.go.kr/analysis#/servicePopulation/adong")

In [ ]:
#구군
select_1 = Select(driver.find_element_by_id('sigCd'))
select_1.select_by_index(2)

In [ ]:
# 행정동 읍/면/동
select_2 = Select(driver.find_element_by_id('adongCd'))
select_2.select_by_index(2)

In [ ]:
#달별로 radio 인풋 선택함
driver.find_element_by_css_selector("input[type='radio'][value='unique']").click()

In [26]:
period = '//*[@id="iptMonth"]'
driver.find_element_by_xpath(period).click()

In [ ]:
#2019로 돌아가는 버튼
before = '/html/body/div[2]/div[2]/table/thead/tr[2]/th[1]'
driver.find_element_by_xpath(before).click()

In [76]:
#월 박스 클릭
period = '//*[@id="iptMonth"]'
driver.find_element_by_xpath(period).click()

In [77]:

month = f'/html/body/div[2]/div[2]/table/tbody/tr/td/span[{n}]'
driver.find_element_by_xpath(month).click()
n+=1

In [50]:
start ='//*[@id="btnAnalysis"]'
driver.find_element_by_xpath(start).click()

In [81]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
infos = soup.select('#gis-result > div > ul > li.tab-sec.active > div > div > div.full-right > div > div > div > table > tbody > tr > td')
row = []
for info in infos:
        add = info.text
        row.append(add) #row 에는 새로운 데이터 원소가 추가 됨.

result = result.append(pd.Series(row,index=result.columns),ignore_index=True)
result.tail(5)

,기준 년월,분석영역,10대,20대,30대,40대,50대,60대
55,201908,중구 대봉1동,"57,435","50,887","59,069","62,541","55,873","83,035"
56,201909,중구 대봉1동,"60,953","51,891","62,123","64,590","56,679","83,995"
57,201910,중구 대봉1동,"61,650","52,589","63,783","65,427","58,497","86,813"
58,201911,중구 대봉1동,"57,491","51,967","62,066","63,839","57,451","83,635"
59,201912,중구 대봉1동,"57,852","53,518","62,904","66,014","59,600","88,355"


In [82]:
#취소버튼
delete = '//*[@id="remove"]'
driver.find_element_by_xpath(delete).click()

In [68]:
col_list = ['기준 년월','분석영역','10대','20대','30대','40대','50대','60대']
result = pd.DataFrame(columns = col_list)

In [113]:
result.to_excel("대구유동인구2019_1.xlsx")

In [87]:
for m in range(8,13):
    
    select_2 = Select(driver.find_element_by_id('adongCd'))
    select_2.select_by_index(f'{m}')
    
    for n in range(1,13):
    #월 박스 클릭
        period = '//*[@id="iptMonth"]'
        driver.find_element_by_xpath(period).click()
    
    #달 선택
        month = f'/html/body/div[2]/div[2]/table/tbody/tr/td/span[{n}]'
        driver.find_element_by_xpath(month).click()

        start ='//*[@id="btnAnalysis"]'
        driver.find_element_by_xpath(start).click()
    
    #부팅이 될 때까지 기다려 주겠니...?
        time.sleep(3)
    
    #스크랩 해서 데이터 프레임에 쏙
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        infos = soup.select('#gis-result > div > ul > li.tab-sec.active > div > div > div.full-right > div > div > div > table > tbody > tr > td')
        row = []
        for info in infos:
            add = info.text
            row.append(add) #row 에는 새로운 데이터 원소가 추가 됨.
            
        result = result.append(pd.Series(row,index=result.columns),ignore_index=True)
    
        driver.implicitly_wait(30)
    
    #취소버튼
        delete = '//*[@id="remove"]'
        driver.find_element_by_xpath(delete).click()
    
        time.sleep(1)
    
    #오류 뜨고나면 새로고침 해줘야 달 선택 가능함. 왜그런지는 모름...

In [88]:
result

,기준 년월,분석영역,10대,20대,30대,40대,50대,60대
0,201901,중구 남산1동,"24,028","71,436","53,451","63,749","68,101","103,139"
1,201902,중구 남산1동,"23,486","67,489","51,397","59,552","63,562","96,285"
2,201903,중구 남산1동,"30,220","70,366","49,685","59,256","62,525","99,310"
3,201904,중구 남산1동,"41,687","31,336","30,553","33,518","35,346","45,088"
4,201905,중구 남산1동,"41,363","31,225","30,479","33,233","34,408","57,613"
...,...,...,...,...,...,...,...,...
139,201908,중구 성내3동,"16,489","19,713","21,818","24,507","31,079","46,768"
140,201909,중구 성내3동,"16,061","19,786","23,289","24,634","29,566","45,061"
141,201910,중구 성내3동,"17,295","22,454","25,596","27,348","32,053","49,828"
142,201911,중구 성내3동,"15,913","21,418","26,263","27,312","32,532","50,999"


In [90]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None
    """
    from openpyxl import load_workbook

    import pandas as pd

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)

        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)

        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()
    
append_df_to_excel("대구유동인구2019.xlsx", result)

In [85]:
for n in range(8,13):
    #월 박스 클릭
        period = '//*[@id="iptMonth"]'
        driver.find_element_by_xpath(period).click()
    
    #달 선택
        month = f'/html/body/div[2]/div[2]/table/tbody/tr/td/span[{n}]'
        driver.find_element_by_xpath(month).click()

        start ='//*[@id="btnAnalysis"]'
        driver.find_element_by_xpath(start).click()
    
    #부팅이 될 때까지 기다려 주겠니...?
        time.sleep(3)
    
    #스크랩 해서 데이터 프레임에 쏙
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        infos = soup.select('#gis-result > div > ul > li.tab-sec.active > div > div > div.full-right > div > div > div > table > tbody > tr > td')
        row = []
        for info in infos:
            add = info.text
            row.append(add) #row 에는 새로운 데이터 원소가 추가 됨.
            
        result = result.append(pd.Series(row,index=result.columns),ignore_index=True)
    
        driver.implicitly_wait(30)
    
    #취소버튼
        delete = '//*[@id="remove"]'
        driver.find_element_by_xpath(delete).click()
    
        time.sleep(1)